In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# Fill in the database you want to work on
working_database = 'Xgridz'

# Fill in your mySQL root pass
my_password = 'dinh9Thuan'

try:
    connection = mysql.connector.connect(host='localhost',
                                         database=working_database,
                                         user='root',
                                         password=my_password)
    if connection.is_connected():
        db_info = connection.get_server_info()
        print('Connected to MySQL Server version ', db_info)
        cursor = connection.cursor()
        
        cursor.execute('select database();')
        record = cursor.fetchone()
        print('Connected to database: ', record)
        
        cursor.execute('show tables;')
        tables = cursor.fetchall()
        print('Tables in {} schema: {}'.format(working_database, tables))
        
        """INSERT DATA TO XGRIDZ"""
        
        # Insert student data from a csv file into student table
        student = pd.read_csv('student.csv')
        Q = ["INSERT INTO student (name, age)\
              VALUES ('{}', '{}');".format(s[0], s[1]) for s in student.values]
        
        for q in Q:
            cursor.execute(q)
        
        q = "SELECT * FROM student;"
        
        cursor.execute(q)
        cursor.fetchall()
        student_table = pd.read_sql_query(q, connection)
        
         # Insert Grid Test 1 data from a csv file into test1 table
        test1 = pd.read_csv('test1.csv')
        Q = ["INSERT INTO test1 (date, correct, total, weight)\
              VALUES ('{}', '{}', '{}', '{}');".\
              format(t[0], t[1], t[2], t[3]) for t in test1.values]
    
        for q in Q:
            cursor.execute(q)
         
        q = "SELECT * FROM test1;"
        
        cursor.execute(q)
        cursor.fetchall()
        test1_table = pd.read_sql_query(q, connection)
        
         # Insert Grid Test 2 data from a csv file into test2 table
        test2 = pd.read_csv('test2.csv')
        Q = ["INSERT INTO test2 (date, correct, total, weight)\
              VALUES ('{}', '{}', '{}', '{}');".\
              format(t[0], t[1], t[2], t[3]) for t in test2.values]
        
        for q in Q:
            cursor.execute(q)
         
        q = "SELECT * FROM test2;"
        
        cursor.execute(q)
        cursor.fetchall()
        test2_table = pd.read_sql_query(q, connection)
        
        
        # Join 3 tables into one

        q = "SELECT test_date, student_ID, name, age,\
            ((correct_1 * weight_1 + correct_2 * weight_2) / (total_1 + total_2)) AS score\
             FROM (SELECT test1.date AS test_date, s.ID AS student_ID, s.name, s.age,\
                          test1.correct AS correct_1, test1.total AS total_1, test1.weight AS weight_1,\
                          test2.correct AS correct_2, test2.total AS total_2, test2.weight AS weight_2 \
                   FROM student AS s\
                               INNER JOIN test1 ON s.ID = test1.student_ID \
                               INNER JOIN test2 ON s.ID = test2.student_ID) sub \
             HAVING score < 0.6;"
        
        cursor.execute(q)
        cursor.fetchall()
        result_table = pd.read_sql_query(q, connection)
        
except Error as e:
    print('Error while connecting to MySQL', e)

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print('\n', '* ' * 40, '\n')
        print('MySQL connection is closed.')

Connected to MySQL Server version  8.0.23
Connected to database:  ('xgridz',)
Tables in Xgridz schema: [('student',), ('test1',), ('test2',)]

 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *  

MySQL connection is closed.


In [2]:
student_table

,ID,name,age
0,100,Koen Frederick,8-9
1,101,Brent Hinton,4-5
2,102,Mitchell Wheeler,4-5
3,103,Pierce Mullen,6-7
4,104,Tara Chase,8-9
5,105,Melany Clay,6-7
6,106,Gavyn Nielsen,4-5
7,107,Anya Ryan,8-9
8,108,Kadin Carey,6-7
9,109,Lyla Alvarado,4-5


In [3]:
test1_table

,student_ID,date,correct,total,weight
0,100,May 29 2020,4,5,1
1,101,May 30 2020,1,5,1
2,102,Aug. 27 2020,4,5,1
3,103,Aug. 11 2020,1,5,1
4,104,Apr. 21 2020,2,5,1
5,105,Oct. 5 2020,3,5,1
6,106,Oct. 2 2020,0,5,1
7,107,Apr. 3 2020,1,5,1
8,108,Nov. 7 2020,1,5,1
9,109,Aug. 29 2020,1,5,1


In [4]:
test2_table

,student_ID,date,correct,total,weight
0,100,May 29 2020,6,12,1
1,101,May 30 2020,7,12,1
2,102,Aug. 27 2020,4,12,1
3,103,Aug. 11 2020,6,12,1
4,104,Apr. 21 2020,4,12,1
5,105,Oct. 5 2020,6,12,1
6,106,Oct. 2 2020,10,12,1
7,107,Apr. 3 2020,3,12,1
8,108,Nov. 7 2020,11,12,1
9,109,Aug. 29 2020,1,12,1


In [5]:
result_table

,test_date,student_ID,name,age,score
0,May 29 2020,100,Koen Frederick,8-9,0.5882
1,May 30 2020,101,Brent Hinton,4-5,0.4706
2,Aug. 27 2020,102,Mitchell Wheeler,4-5,0.4706
3,Aug. 11 2020,103,Pierce Mullen,6-7,0.4118
4,Apr. 21 2020,104,Tara Chase,8-9,0.3529
5,Oct. 5 2020,105,Melany Clay,6-7,0.5294
6,Oct. 2 2020,106,Gavyn Nielsen,4-5,0.5882
7,Apr. 3 2020,107,Anya Ryan,8-9,0.2353
8,Aug. 29 2020,109,Lyla Alvarado,4-5,0.1176
9,Jan. 22 2021,110,Cara Castaneda,4-5,0.3529
